# Modeling

In [16]:
%matplotlib inline
import cPickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC

Grab the engineered data

In [2]:
def read_pickle(file_name):
    f = open(file_name, 'rb')
    p = cPickle.load(f)
    f.close()
    return p


train = read_pickle('data/train.engineered')
test = read_pickle('data/test.engineered')
outcomes = read_pickle('data/outcomes.engineered')
outcomes_le = read_pickle('data/outcomes_le.engineered')

Split the `train` data into training/test sets using the hold-out method. Though there is a DataFrame labeled `test`, this is really the set that we want to make predictions against (and, we don't have labeled examples for this set).

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    np.array(train), outcomes, test_size = 0.2, random_state = 10)

## Baseline model

Though I suspect other models will make more accurate predictions, let me quickly try out a logistic regression model w/ different regularization hyperparameters.

In [7]:
def train_test_model(model, hyperparameters, X_train, X_test, y_train, y_test):
    """
    Given a [model] and a set of possible [hyperparameters], along with 
    matricies corresponding to hold-out cross-validation, returns a model w/ 
    optimized hyperparameters using log-loss scoring and 5-fold cross-validation.
    """
    optimized_model = GridSearchCV(
        model, hyperparameters, cv = 5, n_jobs = -1, scoring = 'log_loss')
    optimized_model.fit(X_train, y_train)
    print 'Optimized parameters:', optimized_model.best_params_
    print 'Log loss:', np.absolute(optimized_model.score(X_test, y_test))
    return optimized_model


def create_submission(name, model, train, outcomes, outcomes_le, test):
    """
    Train [model] on [train] and predict the probabilties on [test], and
    format the submission according to Kaggle.
    """
    clf = model.best_estimator_
    clf.fit(np.array(train), outcomes)
    probs = clf.predict_proba(np.array(test))
    results = pd.DataFrame(probs)
    results.columns = list(outcomes_le.inverse_transform(list(results)))
    results['ID'] = pd.read_csv('data/test.csv')[['ID']].astype(int)
    results = results[['ID', 'Adoption', 'Died', 'Euthanasia', 
                       'Return_to_owner', 'Transfer']]
    results.to_csv('submissions/' + name, index = False)
    return None

In [5]:
%%time
logit_model = train_test_model(
    LogisticRegression(), 
    {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l1', 'l2']}, 
    X_train, X_test, y_train, y_test)

Optimized parameters: {'penalty': 'l1', 'C': 1}
Log loss: 0.88836929837
CPU times: user 12.3 s, sys: 48 ms, total: 12.4 s
Wall time: 2min 2s


/home/jake/miniconda2/envs/shelter-animals/lib/python2.7/site-packages/sklearn/grid_search.py:418: ChangedBehaviorWarning: The long-standing behavior to use the estimator's score function in GridSearchCV.score has changed. The scoring parameter is now used.
  ChangedBehaviorWarning)


In [8]:
create_submission('first_submission.csv', logit_model, 
                  train, outcomes, outcomes_le, test)

My estimate of the test error was much lower than the actual error - the log loss on the public leaderboard for this model is 1.83, compared to 0.89 here. Let's also try a logistic regression using an elastic net penalty instead of an L1 penalty.

In [11]:
%%time
logit_en_model = train_test_model(
    SGDClassifier(penalty = 'elasticnet', loss = 'log'), 
    {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}, 
    X_train, X_test, y_train, y_test)

Optimized parameters: {'alpha': 0.001}
Log loss: 0.896135463298
CPU times: user 2.43 s, sys: 56 ms, total: 2.48 s
Wall time: 10.2 s


The result was a higher error, time to try a more advanced model.

## SVM

I am guessing that a Random Forest will do a better job making predictions than an SVM, but I want to try an SVM first. Fist up is a linear SVM using stochastic gradient descent.

In [19]:
%%time
lin_svm_sgd_model = train_test_model(
    LinearSVC(), 
    {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}, 
    X_train, X_test, y_train, y_test)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/jake/miniconda2/envs/shelter-animals/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/home/jake/miniconda2/envs/shelter-animals/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code without impor